In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=0.0)

Using TensorFlow backend.


In [2]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
   
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract((anchor),(positive))),axis=-1)
    # the (encoding) distance between the anchor and the negative
    neg_dist =tf.reduce_sum(tf.square(tf.subtract((anchor),(negative))),axis=-1)
    # the two previous distances and add alpha.
    basic_loss =(pos_dist-neg_dist+alpha)
    #the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss,0))
    
    return loss

In [6]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [22]:
database = {}

database["Daniel radcliff"] = img_to_encoding("images/Daniel_radcliff.jpg", FRmodel)
database["emma watson"] = img_to_encoding("images/emma watson.jpg", FRmodel)
database["rupert grint"] = img_to_encoding("images/rupert grint.jpg", FRmodel)


In [23]:

def detect_face(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    
    
    ##  the target "encoding" for the image
    encoding = img_to_encoding(image_path,model)
    
    ## Finding the closest encoding ##
    
    # Initializing "min_dist" to a large value
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # L2 distance between the target "encoding" and the current db_enc from the database. 
        dist = np.linalg.norm(encoding-db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. 
        if dist<min_dist:
            min_dist = dist
            identity = name

    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [24]:


face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')


recognizer = cv2.face.LBPHFaceRecognizer_create()

cap = cv2.VideoCapture('C:/Users/diwan/face/saved-media/hp.mp4')

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
        #print(x,y,w,h)
        roi_gray = gray[y:y+h, x:x+w] #(ycord_start, ycord_end)
        roi_color = frame[y:y+h, x:x+w]


            #print(5: #id_)
            #print(labels[id_])
        img_item = "15.jpg"
        cv2.imwrite(img_item, roi_color)
        h=96;
        w=96;
        dim=(h,w)
        i_item = cv2.imread('C:/Users/diwan/face/15.jpg')
        resize=cv2.resize(i_item,dim);
        cv2.imwrite('C:/Users/diwan/face/face_det/resize.jpg',resize)
        dist,identity=detect_face("C:/Users/diwan/face/face_det/resize.jpg", database, FRmodel)
        font = cv2.FONT_HERSHEY_SIMPLEX
        name = str(identity)
        color = (255, 255, 255)
        stroke = 2
        cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

    
        color = (255, 0, 0) #BGR 0-255 
        stroke = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
        #subitems = smile_cascade.detectMultiScale(roi_gray)
        #for (ex,ey,ew,eh) in subitems:
            #cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


Not in the database.
Not in the database.
